In [51]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/loan-data-for-dummy-bank/loan_final313.csv
/kaggle/input/loan-condition-eda-data-cleansing/__results__.html
/kaggle/input/loan-condition-eda-data-cleansing/__output__.json
/kaggle/input/loan-condition-eda-data-cleansing/custom.css
/kaggle/input/loan-condition-eda-data-cleansing/__notebook__.ipynb
/kaggle/input/loan-condition-eda-data-cleansing/df_for_use.pkl
/kaggle/input/loan-condition-eda-data-cleansing/__results___files/__results___11_1.png


### Real Feature in data 

* year : 대출 발생 연도
* issue_d : 대출 발생 일자
* final_d : 마지막 거래일자
* emp_length_int : "근속년수. 0은 1년 미만, 10은 10년 이상"
* home_ownership : "등록 시 대출자에게서 제공된 집 보유 상태. RENT(대여) = 1, OWN(소유) = 2, MORTAGE(담보대출) = 3 "
* home_ownership_cat : ;;
* income_category : "수익 Low = 1, Medium = 2, High = 3 으로 분류"
* annual_inc : 등록시 대출자에게서 제공된 연간 소득
* income_cat : ;;
* loan_amount : 대출금액(달러)
* term : "대출기간(36개월 = 1, 60개월 = 2)"
* term_cat : ;;
* application_type : "개인 대출 신청(=1) 인지, 2명의 대출자에 의해 공동으로 신청된 대출 신청 (=2)인지 여부"
* application_type_cat : ;;
* purpose : 대출이유
* purpose_cat : "대출용도(빛 청산, 카드 대금 결제, 집 개발 등등) 
[credit_card = 1, car = 2, small_business = 3, other = 4, wedding = 5, debt_consolidation =6, 
home_improvement = 7, major_purchase = 8, medical = 9, moving = 10, vacation = 11, house =12,
renewable_energy = 13,  educational = 14]"
* interest_payments : "이자 지불? (Low = 1, High =2 로 분류)"
* interest_payments_cat : ;;
* loan_condition : 대출의 상태(TARGET) (Good Loan = 0 , Bad Loan = 1)
* loan_condition_cat : ;;
* interest_rate : 대출의 이자율
* grade : 대출 등급 ( A ~ G , 1~7)
* grade_cat : ;;
* dti : 금융부채 상환능력을 소득으로 따져서 대출한도를 정하는 계산비율
* total_pymnt : 총 상환금액
* total_rec_prncp : ???
* recoveries : 회수
* installment : 분할 불입금
* region : 거래지역

### Libraries

In [52]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline



In [53]:

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
from sklearn.metrics import f1_score, confusion_matrix, precision_recall_curve, roc_curve
from sklearn.preprocessing import StandardScaler , Binarizer
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import time
import os, sys, gc, warnings, random, datetime
import math
warnings.filterwarnings('ignore')

In [54]:
df = pd.read_pickle("/kaggle/input/loan-condition-eda-data-cleansing/df_for_use.pkl")

In [55]:
df.head()

emp_length_int  home_ownership_cat  annual_inc  income_cat  loan_amount  \
0            10.0                   1       24000           1         5000   
1             0.5                   1       30000           1         2500   
2            10.0                   1       12252           1         2400   
3            10.0                   1       49200           1        10000   
4             1.0                   1       80000           1         3000   

   term_cat  application_type_cat  purpose_cat  interest_payment_cat  \
0         1                     1            1                     1   
1         2                     1            2                     2   
2         1                     1            3                     2   
3         1                     1            4                     2   
4         2                     1            4                     1   

   loan_condition_cat  interest_rate  grade_cat    dti   total_pymnt  \
0                   0          10.65          2  27.65   5861.071414   
1                   1          15.27          3   1.00   1008.710000   
2                   0          15.96          3   8.72   3003.653644   
3                   0          13.49          3  20.00  12226.302210   
4                   0          12.69          2  17.94   3242.170000   

   total_rec_prncp  recoveries  installment  
0          5000.00        0.00       162.87  
1           456.46      117.08        59.83  
2          2400.00        0.00        84.33  
3         10000.00        0.00       339.31  
4          2233.10        0.00        67.79

### Utilities

In [56]:
def get_clf_eval(y_test, pred):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test , pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test,pred)
    f1 = f1_score(y_test, pred)
    auc = roc_auc_score(y_test, pred)
    print('Confusion Matrix')
    print(confusion)
    print('Auccuracy : {0:.4f}, Precision : {1:.4f} , Recall : {2:.4f} , F1_Score : {3:.4f}, ROC_AUC_Score : {4:.4f}'.format(accuracy , precision, recall, f1, auc))
    print('------------------------------------------------------------------------------')

In [57]:
thresholds = {0.3,0.35, 0.4, 0.45, 0.50, 0.55, 0.60}

def get_eval_by_threshold(y_test, pred_proba_c1, thresholds):
    for custom_threshold in thresholds:
        binarizer = Binarizer(threshold = custom_threshold).fit(pred_proba_c1)
        custom_predict = binarizer.transform(pred_proba_c1)
        print('threshold:', custom_threshold)
        get_clf_eval(y_test, custom_predict)

## get_eval_by_threshold(y_test, pred_proba[:,1].reshape(-1,1), thresholds)

### train_test_split (Stratify)

In [58]:
X = df.drop('loan_condition_cat', axis=1)
y = df['loan_condition_cat']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2 , random_state = 2020, stratify = y)



### Random Forest

In [59]:
start = time.time() 

rf_clf = RandomForestClassifier(n_estimators=30, min_samples_leaf=20, max_features=0.7, n_jobs=-1, random_state = 2020, oob_score=True)
rf_clf.fit(X_train, y_train)
rf_roc_score = roc_auc_score(y_test, rf_clf.predict_proba(X_test)[:,1], average = 'macro')
print( 'ROC_AUC : {0:.4f}'.format(rf_roc_score))
# pred_rf = rf_clf.predict(X_test)
# get_clf_eval(y_test, pred_rf)

print("Runtime :", time.time() - start)

ROC_AUC : 0.9125
Runtime : 74.92238545417786


In [60]:
# pred_proba = rf_clf.predict_proba(X_test)
# get_eval_by_threshold(y_test, pred_proba[:,1].reshape(-1,1), thresholds)

### XGBoost

In [61]:
start = time.time()

xgb_clf = XGBClassifier(n_estimators = 1000 , random_state = 2020)
xgb_clf.fit(X_train, y_train, early_stopping_rounds = 100, eval_metric = 'auc', eval_set = [ (X_train,y_train), (X_test, y_test)] , verbose = 50)
xgb_roc_score = roc_auc_score(y_test, xgb_clf.predict_proba(X_test)[:,1], average = 'macro')
print( 'ROC_AUC : {0:.4f}'.format(xgb_roc_score))
# pred_xgb = xgb_clf.predict(X_test)
# get_clf_eval(y_test, pred_xgb)

print("Runtime :", time.time() - start)

[0]	validation_0-auc:0.84005	validation_1-auc:0.83869
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 100 rounds.
[50]	validation_0-auc:0.91022	validation_1-auc:0.90152
[100]	validation_0-auc:0.92669	validation_1-auc:0.91169
[150]	validation_0-auc:0.93428	validation_1-auc:0.91453
[200]	validation_0-auc:0.94047	validation_1-auc:0.91620
[250]	validation_0-auc:0.94513	validation_1-auc:0.91687
[300]	validation_0-auc:0.94914	validation_1-auc:0.91783
[350]	validation_0-auc:0.95273	validation_1-auc:0.91819
[400]	validation_0-auc:0.95647	validation_1-auc:0.91832
[450]	validation_0-auc:0.95932	validation_1-auc:0.91856
[500]	validation_0-auc:0.96223	validation_1-auc:0.91900
[550]	validation_0-auc:0.96484	validation_1-auc:0.91944
[600]	validation_0-auc:0.96728	validation_1-auc:0.91925
[650]	validation_0-auc:0.96912	validation_1-auc:0.91906
Stopping. Best iteration:
[555]	validation_0-auc:0.96519	vali

### LightGBM

In [62]:
import lightgbm as lgb
from lightgbm import LGBMClassifier

In [63]:
start = time.time()

lgbm_clf = LGBMClassifier(n_estimators = 2000)
evals = [(X_test, y_test)]
lgbm_clf.fit(X_train, y_train, early_stopping_rounds = 100, eval_metric = 'auc' , eval_set = evals, verbose = 50)
lgbm_roc_score = roc_auc_score(y_test, lgbm_clf.predict_proba(X_test)[:,1], average = 'macro')
print( 'ROC_AUC : {0:.4f}'.format(lgbm_roc_score))

print("Runtime :", time.time() - start)

Training until validation scores don't improve for 100 rounds
[50]	valid_0's auc: 0.890969	valid_0's binary_logloss: 0.151652
[100]	valid_0's auc: 0.901692	valid_0's binary_logloss: 0.144038
[150]	valid_0's auc: 0.906427	valid_0's binary_logloss: 0.139942
[200]	valid_0's auc: 0.909583	valid_0's binary_logloss: 0.136817
[250]	valid_0's auc: 0.911256	valid_0's binary_logloss: 0.134954
[300]	valid_0's auc: 0.912405	valid_0's binary_logloss: 0.133687
[350]	valid_0's auc: 0.913752	valid_0's binary_logloss: 0.132316
[400]	valid_0's auc: 0.91482	valid_0's binary_logloss: 0.131392
[450]	valid_0's auc: 0.915412	valid_0's binary_logloss: 0.130643
[500]	valid_0's auc: 0.915923	valid_0's binary_logloss: 0.129899
[550]	valid_0's auc: 0.916415	valid_0's binary_logloss: 0.129313
[600]	valid_0's auc: 0.916982	valid_0's binary_logloss: 0.12864
[650]	valid_0's auc: 0.917188	valid_0's binary_logloss: 0.128264
[700]	valid_0's auc: 0.917372	valid_0's binary_logloss: 0.127922
[750]	valid_0's auc: 0.917669	v

### NGBoost

In [64]:
!pip install --upgrade git+https://github.com/stanfordmlgroup/ngboost.git

  Cloning https://github.com/stanfordmlgroup/ngboost.git to /tmp/pip-req-build-l17ypae0
  Running command git clone -q https://github.com/stanfordmlgroup/ngboost.git /tmp/pip-req-build-l17ypae0
     |████████████████████████████████| 340 kB 4.5 MB/s eta 0:00:01
  Created wheel for ngboost: filename=ngboost-0.2.1.dev0-py3-none-any.whl size=34724 sha256=67f16d1182b85e8a9ee2697bf877a2c158057fb54bee7a327c8a074cbc1d3e9a
  Stored in directory: /tmp/pip-ephem-wheel-cache-z8x4mdge/wheels/9a/7f/a2/ce1d8e2bb0b0efd77e813f2311d132f536343daa6c8b5a81ee
  Created wheel for autograd: filename=autograd-1.3-py3-none-any.whl size=47990 sha256=ac15b258fbec0089e1f93daccc2a2af6f4ec3f9cf4cda0c6d1c476ebb5c2a470
  Stored in directory: /root/.cache/pip/wheels/ef/32/31/0e87227cd0ca1d99ad51fbe4b54c6fa02afccf7e483d045e04
Successfully built ngboost autograd


In [65]:
from ngboost import NGBRegressor

### Model with Folds (Stratified 5fold)

In [66]:
X = df.drop('loan_condition_cat', axis=1)
y = df['loan_condition_cat']

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2 , random_state = 2020, stratify = y)

In [67]:
from time import time
from lightgbm import LGBMClassifier

In [68]:
params_lgb={'boosting_type':'gbdt',
           'objective': 'binary',
           'random_state':2020,
           'metric':'auc'}
k_fold=5
kf=StratifiedKFold(n_splits=k_fold,shuffle=True, random_state=2020)
training_start_time = time()
aucs=[]

for fold, (trn_idx,val_idx) in enumerate(kf.split(X,y)):
    start_time = time()
    print('Training on fold {}'.format(fold + 1))
    trn_data = lgb.Dataset(X.iloc[trn_idx], label=y.iloc[trn_idx])
    val_data = lgb.Dataset(X.iloc[val_idx], label=y.iloc[val_idx])
    clf = lgb.train(params_lgb, trn_data, num_boost_round=10000, valid_sets = [trn_data, val_data], 
                    verbose_eval=200, early_stopping_rounds=200)
    aucs.append(clf.best_score['valid_1']['auc'])
    print('Fold {} finished in {}'.format(fold + 1, str(datetime.timedelta(seconds=time() - start_time))))
    
print('-' * 30)
print('Training is completed!.')
print("\n## Mean AUC_Score : ", np.mean(aucs))
print('Total training time is {}'.format(str(datetime.timedelta(seconds=time() - training_start_time))))
# print(clf.best_params_)
print('-' * 30)

Training on fold 1
Training until validation scores don't improve for 200 rounds
[200]	training's auc: 0.921378	valid_1's auc: 0.910363
[400]	training's auc: 0.935012	valid_1's auc: 0.915473
[600]	training's auc: 0.943745	valid_1's auc: 0.917478
[800]	training's auc: 0.950556	valid_1's auc: 0.918829
[1000]	training's auc: 0.956016	valid_1's auc: 0.91951
[1200]	training's auc: 0.960983	valid_1's auc: 0.920172
[1400]	training's auc: 0.965488	valid_1's auc: 0.92028
[1600]	training's auc: 0.969027	valid_1's auc: 0.920362
Early stopping, best iteration is:
[1450]	training's auc: 0.966543	valid_1's auc: 0.920472
Fold 1 finished in 0:02:59.023862
Training on fold 2
Training until validation scores don't improve for 200 rounds
[200]	training's auc: 0.922075	valid_1's auc: 0.91097
[400]	training's auc: 0.935251	valid_1's auc: 0.914974
[600]	training's auc: 0.943957	valid_1's auc: 0.916439
[800]	training's auc: 0.950761	valid_1's auc: 0.917653
[1000]	training's auc: 0.956429	valid_1's auc: 0.918

In [69]:
print("\n## Mean AUC_Score : ", np.mean(aucs))


## Mean AUC_Score :  0.9200908877754816


In [70]:
# y_trainin = y_train.to_frame()
# train_df = pd.concat([X_train, y_trainin], axis=1)
# y_testet = y_test.to_frame()
# test_df = pd.concat([X_test, y_testet], axis=1)


# TARGET = 'loan_condition_cat'

In [71]:
rf_clf = RandomForestClassifier(n_estimators=30, min_samples_leaf=20, max_features=0.7, n_jobs=-1, random_state = 2020, oob_score=True)

In [ ]:
# def exec_StratifiedKFold(clf, folds = 5):
#     skfold = StratifiedKFold(n_splits = folds)
#     n_iter = 0
#     cv_auc_scores = []
#     scores = []
    
#     #execute StratifiedKFold
    
#     for train_index , test_index in skfold.split(X,y):
#         X_train , X_test = X.loc[train_index], X.loc[test_index]
#         y_train , y_test = y[train_index], y[test_index]
        
#         # Train, predict, auc score from Classifier
#         clf.fit(X_train, y_train)
#         predictions = clf.predict(X_test)
        
#         n_iter += 1
#         auc_score = np.round(roc_auc_score(y_test, predictions) ,4)
#         train_size = X_train.shape[0]
#         test_size = X_test.shape[0]
#         print("\n#{0} Cross Validation AUC Score : {1} , Size of train data : {2}, Size of test data : {3}".format(n_iter , auc_score,train_size,test_size))
#         cv_auc_scores.append(auc_score)
        
#     # Calculate mean score from 5-fold
#     print("\n## CV AUC_Score: ", np.round(cv_auc_scores,4))
#     mean_score = np.mean(scores)
#     print("\n## Mean AUC_Score : ", np.mean(cv_auc_scores))

        
# exec_StratifiedKFold(rf_clf, folds = 5)